In [1]:
import cv2
import numpy as np

In [2]:
points = []

def select_corner(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 4:
            points.append([x, y])
            print(f"Point {len(points)}: ({x}, {y})")
            # Draw a small circle where the user clicked
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)

In [3]:
def sort_points(pts):
    pts = np.array(pts, dtype="float32")
    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]     # top-left
    rect[2] = pts[np.argmax(s)]     # bottom-right

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect

In [4]:
def alignImages(img,pts):
 
 # Destination A4 size (2480x3508 for 72 DPI)
    width, height = 595, 842
    dst_pts = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype=np.float32)

    # Find homography
    h, _ = cv2.findHomography(pts, dst_pts, cv2.RANSAC)
 
  # Use homography
    imReg = cv2.warpPerspective(img, h, (width, height))
    
    return imReg

In [5]:
img = cv2.resize(cv2.imread("test_img.jpg", cv2.IMREAD_COLOR),(400,600))

# selecting corner points
cv2.namedWindow("camscanner")
cv2.setMouseCallback("camscanner", select_corner)

while True:
    cv2.imshow("camscanner", img)
    key = cv2.waitKey(1) & 0xFF
    if key == 13:
        cv2.destroyAllWindows()
        # Aligning images
        imReg = alignImages(img,sort_points(points))
        out_img=cv2.resize(imReg,(400,600))
        cv2.imwrite("output.jpg",out_img) # Write aligned image to disk.
        cv2.imshow("preview",out_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
    # ESC to exit
    if key == 27:
        print("Scanning Cancelled")
        cv2.destroyAllWindows()
        break

Point 1: (130, 135)
Point 2: (333, 155)
Point 3: (250, 521)
Point 4: (26, 425)
